In [6]:
#Fine-Tuning BERT with a Logistic Regression Layer
import torch
from torch import nn
from transformers import Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
dataset = load_dataset("imdb")

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [9]:
import torch
from transformers import Trainer, TrainingArguments, DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load IMDB dataset
dataset = load_dataset("imdb")

# Load tokenizer and model (smaller model: DistilBERT)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Reduce dataset size for light training
small_train_dataset = tokenized_datasets["train"].select(range(500))
small_eval_dataset = tokenized_datasets["test"].select(range(200))

# Update training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    max_steps=500,
    no_cuda=True,  # Disable GPU usage
)

# Function to extract embeddings
def extract_embeddings(model, dataset):
    embeddings = []
    labels = []
    model.eval()
    with torch.no_grad():
        for batch in dataset:
            input_ids = torch.tensor(batch["input_ids"])
            attention_mask = torch.tensor(batch["attention_mask"])
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits.cpu().numpy()
            embeddings.extend(logits)
            labels.extend(batch["label"])
    return embeddings, labels

# Extract embeddings and train logistic regression
train_embeddings, train_labels = extract_embeddings(model, small_train_dataset)
test_embeddings, test_labels = extract_embeddings(model, small_eval_dataset)

log_reg = LogisticRegression(max_iter=500)
log_reg.fit(train_embeddings, train_labels)

train_preds = log_reg.predict(train_embeddings)
test_preds = log_reg.predict(test_embeddings)

train_accuracy = accuracy_score(train_labels, train_preds)
test_accuracy = accuracy_score(test_labels, test_preds)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

# Initialize Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)

trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/j/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/j/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1583: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


IndexError: tuple index out of range

In [ ]:
# Install Hugging Face Transformers library if not already installed
# !pip install transformers

from transformers import DistilBertTokenizer, DistilBertModel

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Load the model
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

# Example text input
text = "Quantitative finance relies heavily on risk modeling."

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Pass tokenized input through the model
outputs = model(**inputs)

# Print the model's output
print("Last Hidden States Shape:", outputs.last_hidden_state.shape)


In [ ]:
outputs